In [54]:
%matplotlib inline
import pandas as pd
import numpy as np
import torch
from d2l import torch as d2l

class KaggleHouse(d2l.DataModule):
    def __init__(self, batch_size, train = None, val = None):
        super().__init__()
        self.save_hyperparameters()
        if self.train is None:
            self.raw_train = pd.read_csv(d2l.download(
                d2l.DATA_URL + 'kaggle_house_pred_train.csv', self.root,
                sha1_hash='585e9cc93e70b39160e7921475f9bcd7d31219ce'))
            self.raw_val = pd.read_csv(d2l.download(
                d2l.DATA_URL + 'kaggle_house_pred_test.csv', self.root,
                sha1_hash='fa19780a7b011d9b009e8bff8e99922a8ee2eb90'
            ))

data = KaggleHouse(batch_size=64)
print(data.raw_train.shape)
print(data.raw_val.shape)
print(data.raw_train.iloc[:4,[0, 1, 2, 3, -3, -2, -1]])

(1460, 81)
(1459, 80)
   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


数据预处理

In [55]:
# @d2l.add_to_class(KaggleHouse)
# def preprocess(self):
#     label = 'SalePrice'
#     features = pd.concat(
#         (self.raw_train.drop(columns=['Id', label]),
#          self.raw_val.drop(columns=['Id']))
#     )
#     # 标准化数值列
#     numeric_features = features.dtypes[features.dtypes != 'object'].index
#     # 使用均值来填充 NaN 值
#     features[numeric_features] = features[numeric_features].fillna(
#         features[numeric_features].mean()
#     )
#     features[numeric_features] = features[numeric_features].apply(
#         lambda x: (x - x.mean()) / (x.std())
#     )
#     # # 使用 0 来填充 NAN 值
#     # features[numeric_features] = features[numeric_features].fillna(0)
#     # 使用独热编码替换分类变量
#     features = pd.get_dummies(features, dummy_na=True)
#     self.train = features[:self.raw_train.shape[0]].copy()
#     self.train[label]=self.raw_train[label]
#     self.val = features[self.raw_train.shape[0]:].copy()

@d2l.add_to_class(KaggleHouse)
def preprocess(self):
    label = 'SalePrice'

    # 1. 分离训练集和测试集（避免数据泄露）
    train_features = self.raw_train.drop(columns=['Id', label])
    val_features = self.raw_val.drop(columns=['Id'])

    # 2. 识别数值特征（仅用于插补）
    numeric_features = train_features.dtypes[train_features.dtypes != 'object'].index

    # 3. 使用KNN插补（仅用训练集拟合，用于训练集和测试集）
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=5)  # K=5是常用值

    # 用训练集拟合插补器
    train_imputed = imputer.fit_transform(train_features[numeric_features])
    val_imputed = imputer.transform(val_features[numeric_features])

    # 4. 将插补后的数值特征替换回DataFrame
    train_features[numeric_features] = train_imputed
    val_features[numeric_features] = val_imputed

    # 5. 合并处理后的特征（现在所有数值特征已无缺失）
    features = pd.concat([train_features, val_features])

    # 6. 标准化数值特征（使用训练集的均值和标准差）
    for col in numeric_features:
        mean = train_features[col].mean()
        std = train_features[col].std()
        features[col] = (features[col] - mean) / std

    # 7. 处理分类特征（保留NaN作为独立类别）
    features = pd.get_dummies(features, dummy_na=True)

    # 8. 拆分回训练集和测试集
    n_train = self.raw_train.shape[0]
    self.train = features.iloc[:n_train].copy()
    self.train[label] = self.raw_train[label]
    self.val = features.iloc[n_train:].copy()

data.preprocess()
print(data.train.shape)
print(data.train.iloc[:4,[0, 1, 2, 3, -3, -2, -1]])

(1460, 331)
   MSSubClass  LotFrontage   LotArea  OverallQual  SaleCondition_Partial  \
0    0.073350    -0.250128 -0.207071     0.651256                  False   
1   -0.872264     0.381377 -0.091855    -0.071812                  False   
2    0.073350    -0.123827  0.073455     0.651256                  False   
3    0.309753    -0.460630 -0.096864     0.651256                  False   

   SaleCondition_nan  SalePrice  
0              False     208500  
1              False     181500  
2              False     223500  
3              False     140000  


定义均方对数误差

In [56]:
@d2l.add_to_class(KaggleHouse)
def get_dataloader(self, train):
    label = 'SalePrice'
    data = self.train if train else self.val
    if label not in data: return
    get_tensor = lambda x: torch.tensor(x.values.astype(float),
                                        dtype=torch.float32)
    tensors = (get_tensor(data.drop(columns=[label])),
               torch.log(get_tensor(data[label])).reshape((-1, 1)))
    return self.get_tensorloader(tensors, train)

K折交叉验证

In [57]:
# def k_fold_data(data, k):
#     rets = []
#     fold_size = data.train.shape[0] // k
#     for j in range(k):
#         idx = range(j * fold_size, (j + 1) * fold_size)
#         rets.append(KaggleHouse(data.batch_size, data.train.drop(index=idx), data.train.loc[idx]))
#
#     return rets
def k_fold_data(data, k):
    rets = []
    n = len(data.train)

    # 打乱索引
    indices = np.random.permutation(n)

    fold_size = n // k
    remainder = n % k

    start = 0
    for j in range(k):
        # 当前 fold 大小：前 remainder 个 fold 多一个样本
        current_size = fold_size + (1 if j < remainder else 0)
        end = start + current_size

        val_indices = indices[start:end]
        train_indices = np.concatenate([indices[:start], indices[end:]])

        val_df = data.train.iloc[val_indices].copy()
        train_df = data.train.iloc[train_indices].copy()

        rets.append(KaggleHouse(data.batch_size, train_df, val_df))
        start = end

    return rets

def k_fold(trainer, data, k, lr):
    val_loss, models = [], []
    for i, data_fold in enumerate(k_fold_data(data, k)):
        model = d2l.LinearRegression(lr)
        model.board.yscale = 'log'
        if i != 0: model.board.display = False
        trainer.fit(model, data_fold)
        val_loss.append(float(model.board.data['val_loss'][-1].y))
        models.append(model)
    print(f'average validation log mse = {sum(val_loss)/len(val_loss)}')
    return models

训练

In [58]:
trainer = d2l.Trainer(max_epochs=150)
models = k_fold(trainer, data, k = 5, lr = 0.01)

average validation log mse = 0.03832870535552502


In [60]:
preds = [model(torch.tensor(data.val.values.astype(float), dtype=torch.float32)) for model in models]
ensemble_preds = torch.exp(torch.cat(preds, 1)).mean(1)
submission = pd.DataFrame({'Id': data.raw_val.Id, 'SalePrice': ensemble_preds.detach().numpy()})
submission.to_csv('submission.csv', index=False)